## VGG16 Test
This Test takes too much time (approx 5.5h).

In [ ]:
import sys
import os

# Annahme: Das Skript wird im results-Ordner ausgeführt
current_dir = os.getcwd()

# Pfad zum src-Ordner zum Python-Pfad hinzufügen
src_path = os.path.abspath(os.path.join(current_dir, '..', 'src'))
sys.path.append(src_path)

# Jetzt kannst du die Module direkt aus dem src-Ordner importieren
import numpy as np
from filter_testing import *
from custom_filters import *

In [ ]:
import torch
import torch.optim as optim
import torch.nn as nn
import torchvision.models as models
from torchsummary import summary

from utils import fit, train_model, test_model
from synthetic_data_generation import generate_train_test_valid_data
from filter_testing import plot_graph

dataset = np.load("../data/256x256_ml_ppa_data_binary.npz")
labels = dataset["labels"]
file = dataset.files[2]
data = dataset[file]
noise = int(file.split("_")[1])

bs = 16
epochs = 50
train_data, test_data = generate_train_test_valid_data(data, labels, False, bs, 0.2)

# Default Filters
vgg16_default = models.vgg16(weights=None)
vgg16_default.features[0] = nn.Conv2d(in_channels=1, out_channels=64, kernel_size=3, padding=1)

num_classes = 2 
vgg16_default.classifier[6] = nn.Linear(in_features=4096, out_features=num_classes)

lr = 0.001
opt_default = optim.Adam(vgg16_default.parameters(), lr=lr)
loss_fun = nn.CrossEntropyLoss()
_, _, _, def_test_acc, _ = fit(vgg16_default, epochs, opt_default, loss_fun, train_data, test_data, learn_plot=False)


# Custom Filters

# Adjust to 256x256 data with 2 classes
vgg16_custom = models.vgg16(weights=None)
vgg16_custom.features[0] = nn.Conv2d(in_channels=1, out_channels=64, kernel_size=3, padding=1)
vgg16_custom.classifier[6] = nn.Linear(in_features=4096, out_features=2)

# Add layer for custom filters
prewitt_x = prewitt_filter("x")
prewitt_y = prewitt_filter("y")

custom_filter_list = [prewitt_x, prewitt_y]

filters = torch.from_numpy(np.array([[f] for f in custom_filter_list], dtype=np.float32))
custom_filter_conv = nn.Conv2d(1, len(filters), kernel_size=3, padding=1, bias=False)
custom_filter_conv.weight = nn.Parameter(filters)

original_features = vgg16_custom.features

vgg16_custom.features = nn.Sequential(
    custom_filter_conv,
    *original_features
)

lr = 0.001
opt_custom = optim.Adam(vgg16_custom.parameters(), lr=lr)
loss_fun = nn.CrossEntropyLoss()
_, _, _, cust_test_acc, _ = fit(vgg16_custom, epochs, opt_custom, loss_fun, train_data, test_data, learn_plot=False)

plot_graph(epochs, noise, def_test_acc, cust_test_acc, lr, bs)
